# Generate Job

In [1]:
RATE = 1.3

In [2]:
EXAMPLE_JOB = (
    {
        "name" : 'eagle',
        "job" : [{'3DP': 300}] # single job
    },
    {
        "name" : 'box',
        "job" : ([{'3DP': 300}], [{'3DP': 300}]) # parallel job
    },
    {
        "name" : 'clock',
        "job" : [ # job shop
            (
                (
                    [{'3DP': 200},{'CNC':10}],
                    [{'3DP': 200},{'CNC':5}]
                ),
                [{'3DP': 100}]
            ),
            {'CNC' : 10},
            {'Vapor' : 5}
        ]
    }
)

In [3]:
import random

In [4]:
def gen_job(single_jobs=10, parallel_jobs=10, job_shops=5):
    ret = []
    job_names = set()
    for i in range(single_jobs):
        ret.append({
            'name' : 'single' + str(i),
            'job' : [{'3DP' : int(random.randint(1*60,3*60) * RATE)}]
        })
        job_names.add('single' + str(i))
    for i in range(parallel_jobs):
        ret.append({
            'name' : 'parallel' + str(i),
            'job' : ([{'3DP' : int(random.randint(1*60,3*60) * RATE)}],[{'3DP' : int(random.randint(1*60,3*60) * RATE)}])
        })
        job_names.add('parallel' + str(i))
    for i in range(job_shops):
        ret.append( {
            "name" : 'clock' + str(i),
            "job" : [ # job shop
                (
                    (
                        [{'3DP': int(random.randint(1*60,3*60) * RATE) },{'CNC':int(random.randint(10,30) * RATE)}],
                        [{'3DP': int(random.randint(1*60,3*60) * RATE) },{'CNC':int(random.randint(10,30) * RATE)}]
                    ),
                    [{'3DP': int(random.randint(1*60,3*60) * RATE)}]
                ),
                {'CNC' : int(random.randint(10,30) * RATE)},
                {'Vapor' : int(random.randint(10,30) * RATE)}
            ]
        })
        job_names.add('clock' + str(i))
    return ret, job_names

In [5]:
lit, job_names= gen_job()

In [6]:
def ret_list():
    return tuple(lit)

# append to factory

In [7]:
import factory_env

In [8]:
DAILY_CAPA = 8 * 60

In [9]:
env = factory_env.FactoryEnv(generate_job_func=ret_list, machine_map=
                             {'3DP': (12, DAILY_CAPA), 'CNC':(1,DAILY_CAPA),'Vapor':(1,DAILY_CAPA)}, 
                             job_type='single', job_slot_num=5)

In [10]:
env.generate_job()

({'job': [{'3DP': 185}], 'name': 'single0'},
 {'job': [{'3DP': 182}], 'name': 'single1'},
 {'job': [{'3DP': 159}], 'name': 'single2'},
 {'job': [{'3DP': 150}], 'name': 'single3'},
 {'job': [{'3DP': 202}], 'name': 'single4'},
 {'job': [{'3DP': 171}], 'name': 'single5'},
 {'job': [{'3DP': 79}], 'name': 'single6'},
 {'job': [{'3DP': 152}], 'name': 'single7'},
 {'job': [{'3DP': 182}], 'name': 'single8'},
 {'job': [{'3DP': 148}], 'name': 'single9'},
 {'job': ([{'3DP': 131}], [{'3DP': 101}]), 'name': 'parallel0'},
 {'job': ([{'3DP': 136}], [{'3DP': 148}]), 'name': 'parallel1'},
 {'job': ([{'3DP': 174}], [{'3DP': 92}]), 'name': 'parallel2'},
 {'job': ([{'3DP': 166}], [{'3DP': 161}]), 'name': 'parallel3'},
 {'job': ([{'3DP': 193}], [{'3DP': 167}]), 'name': 'parallel4'},
 {'job': ([{'3DP': 187}], [{'3DP': 161}]), 'name': 'parallel5'},
 {'job': ([{'3DP': 228}], [{'3DP': 135}]), 'name': 'parallel6'},
 {'job': ([{'3DP': 153}], [{'3DP': 202}]), 'name': 'parallel7'},
 {'job': ([{'3DP': 218}], [{'3DP

# Online planning

In [11]:
import numpy as np

In [12]:
def FIFO(state, job_slot):
    ## return action
    selected_job_slot = 0
    # select done job
    for job_slot_num, slot in enumerate(job_slot):
        if slot != [] and sum(state[-1][job_slot_num][1:]) != 0:
            selected_job_slot = job_slot_num
            break
    # find allocate machine
    job_capa = state[-1][job_slot_num][0]
    job_slot_state = state[-1][job_slot_num][1:]
    select_machine_num = 0
    for count, i in enumerate(job_slot_state):
        if i == 1:
            select_machine_num = count
            break
    
    action_space = np.zeros([len(job_slot), len(state[-1][0][1:])])
    action_space[selected_job_slot][select_machine_num] = 1
    
    return action_space, selected_job_slot, select_machine_num

In [13]:
env.job_slots

[[[185, '3DP', 'single0_0'],
  [182, '3DP', 'single8_0'],
  [174, '3DP', 'parallel2_0'],
  [187, '3DP', 'parallel5_0'],
  [202, '3DP', 'parallel7_0'],
  [105, '3DP', 'clock0_0'],
  [28, 'CNC', 'clock0_0'],
  [145, '3DP', 'clock2_0'],
  [13, 'CNC', 'clock2_0'],
  [87, '3DP', 'clock2_1'],
  [15, 'CNC', 'clock2_2'],
  [28, 'Vapor', 'clock2_2']],
 [[182, '3DP', 'single1_0'],
  [152, '3DP', 'single7_0'],
  [136, '3DP', 'parallel1_0'],
  [193, '3DP', 'parallel4_0'],
  [135, '3DP', 'parallel6_0'],
  [191, '3DP', 'parallel8_0'],
  [195, '3DP', 'clock2_0'],
  [22, 'CNC', 'clock2_0'],
  [88, '3DP', 'clock3_0'],
  [13, 'CNC', 'clock3_0'],
  [78, '3DP', 'clock3_1'],
  [29, 'CNC', 'clock3_2'],
  [14, 'Vapor', 'clock3_2']],
 [[159, '3DP', 'single2_0'],
  [79, '3DP', 'single6_0'],
  [131, '3DP', 'parallel0_0'],
  [92, '3DP', 'parallel2_0'],
  [161, '3DP', 'parallel3_0'],
  [228, '3DP', 'parallel6_0'],
  [104, '3DP', 'parallel9_0'],
  [105, '3DP', 'clock0_0'],
  [37, 'CNC', 'clock0_0'],
  [87, '3DP', 

In [14]:
while not env._goal_test():
    env._next_state(FIFO(env.get_state(), env.job_slots)[0])

In [15]:
env.get_state()

[425,
 289,
 8,
 32,
 45,
 10,
 73,
 2,
 29,
 43,
 40,
 143,
 101,
 74,
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [78, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [140, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [148, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]]

In [16]:
def check_unfinished_job(slots):
    unfinished_job = set()
    for job_slot in slots:
        for wait in job_slot:
            unfinished_job.add(wait[-1].split('_')[0])
    return unfinished_job

In [17]:
check_unfinished_job(env.job_slots)

{'clock1',
 'clock3',
 'clock4',
 'parallel1',
 'parallel4',
 'parallel7',
 'parallel9',
 'single9'}

In [18]:
ret_li = []

for i in range(100000):
    lit, job_names= gen_job()
    def ret_list():
        return tuple(lit)
    env = factory_env.FactoryEnv(generate_job_func=ret_list, machine_map=
                             {'3DP': (12, DAILY_CAPA), 'CNC':(1,DAILY_CAPA),'Vapor':(1,DAILY_CAPA)}, 
                             job_type='single', job_slot_num=10)
    while not env._goal_test():
        env._next_state(FIFO(env.get_state(), env.job_slots)[0])
    unfinished_job = check_unfinished_job(env.job_slots)
    ret_li.append([{'3DP': (12, DAILY_CAPA), 'CNC':(1,DAILY_CAPA),'Vapor':(1,DAILY_CAPA)}, lit, job_names, unfinished_job])
    if i%10000 == 0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000


In [19]:
import pickle

In [20]:
with open('plan_list.pkl', 'wb') as f:
    pickle.dump(ret_li, f)

In [21]:
with open('plan_list.pkl', 'rb') as f:
    plan_results = pickle.load(f)